# Import

In [1]:
import os
import json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import sklearn.metrics as metrics
import lightgbm as lgb
import shap

import gradio as gr

# ModelPredictor

In [ ]:
def plot_roc_curve(y_true, y_score, figsize=(5,5)):
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score)
    roc_auc = metrics.auc(fpr, tpr)
    plt.figure(figsize=figsize)
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(fpr, tpr, color='darkorange', label=f"ROC Curve (AUC = {roc_auc:.3f})")
    ax.plot([0,1], [0,1], color='navy', linestyle='--')
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title("ROC Curve")
    ax.legend(loc="lower right")
    return fig


def plot_confusion_matrix(y_true, y_pred, figsize=(6, 5)):
    fig, ax = plt.subplots(figsize=figsize)
    cm = metrics.confusion_matrix(y_true, y_pred)
    disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Not Default', 'Default'])
    disp.plot(cmap=plt.cm.Greens, ax=ax, values_format='d')
    
    cm_normalized = cm.astype('float') / np.full((2, 2), cm.sum())
    # Overlay percentages as annotations
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            percentage = cm_normalized[i, j] * 100
            ax.text(j, i + 0.2, f'{percentage:.1f}%', ha='center', va='center', color='red', fontsize=10)
    ax.set_title('Confusion Matrix')
    return fig

def get_classification_report(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    return pd.DataFrame([
        ("n_sample", y_true.shape[0]),
        ("n_default", y_true.sum()),
        ("percent_default", round(y_true.mean() * 100, 2)),
        ("Accuracy", round(metrics.accuracy_score(y_true, y_pred), 2)),
        ("Precision", round(metrics.precision_score(y_true, y_pred), 2)),
        ("Recall", round(metrics.recall_score(y_true, y_pred), 2)),
        ("F1-score", round(metrics.f1_score(y_true, y_pred), 2))
    ], columns=["Metric", "Value"])

In [ ]:
class ModelPredictor:
    def __init__(self, model_dir="models/"):
        self.model_dir = Path(model_dir)
        self.models = self.load_models()
        self.config = json.load(open(self.model_dir / "model_config.json"))

    # -----------------------------------------
    def load_models(self):
        return [lgb.Booster(model_file=self.model_dir / f"model_{i}.booster") for i in range(5)]

    # -----------------------------------------
    def preprocess_data(self, df: pd.DataFrame):
        df = df.copy()
        cat_features = self.config["cat_features"]
        df[cat_features] = df[cat_features].astype(str).astype("category")
        return df

    # -----------------------------------------
    def run_prediction(self, df: pd.DataFrame):
        df = self.preprocess_data(df)
        final_score = np.mean([model.predict(df[self.config["features"]]) for model in self.models], axis=0)
        final_pred = (final_score >= self.config["threshold"]).astype(int)
        return final_score, final_pred

    # -----------------------------------------
    def plot_shap_feature_importance(self, df, figsize=(5, 8), model_index=0, max_n_sample=1000):
        df = self.preprocess_data(df)
        X = df[self.config["features"]].head(max_n_sample)
        explainer = shap.TreeExplainer(self.models[model_index])
        shap_values = explainer.shap_values(X)
        fig = plt.figure()  # figsize=figsize
        shap.summary_plot(shap_values, X, plot_type='dot', show=False)
        return fig

    # -----------------------------------------
    def plot_shap_waterfall(self, df, model_index=0):
        df = self.preprocess_data(df)
        X = df[self.config["features"]]
        explainer = shap.TreeExplainer(self.models[model_index])
        shap_values = explainer(X)
        fig, ax = plt.subplots()
        shap.plots.waterfall(shap_values[0], show=False)
        plt.tight_layout()
        return fig

    # -----------------------------------------
    def run_evaluation(self, df):
        y_score, y_pred = self.run_prediction(df)
        y_true = df[self.config["target"]]
        roc = plot_roc_curve(y_true, y_score, figsize=(5,5))
        cm = plot_confusion_matrix(y_true, y_pred, figsize=(5, 4))
        cr = get_classification_report(y_true, y_pred)
        fi = self.plot_shap_feature_importance(df, figsize=(5, 8))
        return roc, cm, cr, fi

In [ ]:
def plot_eda_charts(df):
    # -----------------------------
    #  Chart 1 — Histogram score
    # -----------------------------
    fig1, ax1 = plt.subplots()
    ax1.hist(df["pred_score"], bins=20)
    ax1.set_title("Histogram of Predicted Scores")
    ax1.set_xlabel("Score")
    ax1.set_ylabel("Count")

    # -----------------------------
    #  Chart 2 — Pie chart label
    # -----------------------------
    fig2, ax2 = plt.subplots()
    label_counts = df["pred_label"].value_counts()
    ax2.pie(label_counts, labels=label_counts.index, autopct='%1.1f%%')
    ax2.set_title("Predicted Label Distribution")

    # -----------------------------
    #  Chart 3 — Weekly default rate
    # -----------------------------
    df["week"] = df["request_date"].dt.to_period("W").apply(lambda r: r.start_time)

    weekly = df.groupby("week")["pred_label"].mean().reset_index()
    weekly["pred_label"] *= 100  # chuyển sang %

    fig3, ax3 = plt.subplots()
    ax3.plot(weekly["week"], weekly["pred_label"], marker="o")
    ax3.set_title("Weekly Predicted Default Rate (%)")
    ax3.set_ylabel("% label = 1")
    ax3.set_xlabel("Week")
    plt.xticks(rotation=30)
    
    return fig1, fig2, fig3

# UI Login

In [ ]:
USERS = {
    "officer": {"pwd": "123", "role": "Loan Officer"},
    "risk": {"pwd": "123", "role": "Risk Manager"},
    "ds": {"pwd": "123", "role": "Data Scientist"},
    "user": {"pwd": "123", "role": "End User"}
}

def login(username, password):
    user = USERS.get(username)
    if user and user["pwd"] == password:
        return f"✅ Đăng nhập thành công ({user['role']})", user["role"], gr.update(visible=False)
    return "❌ Sai username hoặc password", None, gr.update(visible=True)

def show_tabs(role):
    return (
        gr.update(visible=(role == "Loan Officer")),
        gr.update(visible=(role == "Risk Manager")),
        gr.update(visible=(role == "Data Scientist")),
        gr.update(visible=(role == "End User")),
    )

# UI Theme

In [ ]:
# -------------------------
# CSS & Theme
# -------------------------
# Light, modern palette with teal accents and warm accent color
CSS = """
:root{
  --bg:#f7fbfc;
  --card:#ffffff;
  --muted:#6b7280;
  --accent:#0e9aa7;    /* teal */
  --accent-2:#ff8a4c;  /* warm orange */
  --panel-border: rgba(16,24,40,0.06);
}
.gradio-container { background: var(--bg); color: #0f172a; font-family: Inter, system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", Arial; padding: 16px; }
/* Card style for blocks */
.gr-box, .gr-panel, .gr-form, .gr-group { background: var(--card) !important; border: 1px solid var(--panel-border) !important; border-radius: 12px !important; padding: 14px !important; box-shadow: 0 6px 18px rgba(16,24,40,0.04); }
/* Headings */
.prose h1, .prose h2 { color: #0f172a !important; }
.prose p, .prose small { color: var(--muted) !important; }
/* Buttons */
button.primary { background: var(--accent) !important; color: white !important; border-radius: 10px !important; padding: 8px 12px !important; box-shadow: 0 6px 12px rgba(14,154,167,0.18); }
button.secondary { background: var(--accent-2) !important; color: white !important; border-radius: 10px !important; padding: 8px 12px !important; }
/* Inputs */
input, textarea, select { border-radius: 8px !important; border: 1px solid rgba(15,23,42,0.06) !important; padding: 8px !important; }
/* Small labels */
.label-wrap .label { color: var(--muted) !important; font-weight:600; }
/* Tables / Dataframe */
.dataframe thead th { background: transparent !important; color: #0f172a !important; font-weight:700; }
.dataframe tbody tr:nth-child(even) { background: #fbfcfd !important; }
/* Tabs */
.tabs { background: transparent !important; padding-bottom: 0; }
.tabitem.selected { border-bottom: 3px solid var(--accent) !important; }
"""

# Main

In [ ]:
predictor = ModelPredictor(model_dir="../models/v2025-11-27")

In [ ]:
# -------------------------
# Build the Gradio app
# -------------------------
with gr.Blocks(title="Loan Default Prediction", css=CSS) as app:
    
    # Header
    with gr.Row(elem_id="header-row"):
        gr.Markdown(
            """
            # Loan Default Prediction
            """)
        with gr.Column(min_width=180):
            gr.Markdown("**Status:** Demo • no production data")
            gr.Button("Tải hướng dẫn (PDF)", variant="secondary")

    
    # =========== LOGIN BOX ============
    gr.Markdown("## 🔐 Đăng nhập hệ thống")
    login_box = gr.Group(visible=True)
    with login_box:
        username = gr.Textbox(label="Username")
        password = gr.Textbox(label="Password", type="password")
        login_btn = gr.Button("Đăng nhập")
    
    login_status = gr.Markdown(label="Trạng thái")
    role_state = gr.State()
    
    gr.Markdown("---")

    with gr.Tabs():
        # -----------------------
        # Loan Officer Tab
        # -----------------------
        with gr.TabItem("Loan Officer", visible=False) as tab_officer:
            with gr.Row():
                # Left column: search + quick card
                with gr.Column():
                    gr.Markdown("#### Tìm hồ sơ / Tra cứu nhanh")
                    upload_eval_btn = gr.File(label="Upload danh sách hồ sơ", file_types=[".csv"])
                    profile = gr.Dropdown(choices=[], label="Chọn LoanID", value=None)
                    profile_info = gr.DataFrame(label="Thông tin hồ sơ", interactive=False)
                    df_state = gr.State()  # Save df into a Gradio state variable
                    
                    with gr.Row():
                        decision = gr.Radio(choices=["Phê duyệt", "Phê duyệt có điều kiện", "Từ chối"], label="Quyết định")
                    
                    decision_feedback = gr.Markdown("*(Chưa có quyết định được chọn)*")
                    
                    def load_csv(file):
                        if file is None:
                            return gr.Dropdown.update(choices=[]), None
                        df = pd.read_csv(file.name).head(1000)  # limit 1000 for demo
                        loan_ids = df["LoanID"].dropna().astype(str).unique().tolist()
                        return gr.update(choices=loan_ids, value=None), df

                    upload_eval_btn.upload(fn=load_csv, inputs=upload_eval_btn, outputs=[profile, df_state])
                    
                    def on_decision_change(decision_choice):
                        return f"**Đã lưu: {decision_choice}**"
                    
                    decision.change(fn=on_decision_change, inputs=[decision], outputs=[decision_feedback])

                # Right column: score + reasons
                with gr.Column():
                    gr.Markdown("#### Kết quả dự đoán")
                    with gr.Row():
                        score_box = gr.Textbox(label="Xác suất Default (0,1)", interactive=False)
                        decision_box = gr.Textbox(label="Gợi ý quyết định", interactive=False)
                    expl_plot = gr.Plot(label="Yếu tố ảnh hưởng")
                    
                    def lookup_fn(loan_id, df):
                        if df is None or loan_id is None:
                            return [None] * 4
                        res = df.query(f"LoanID == '{loan_id}'")
                        if res.empty:
                            return [None] * 4
                        score, pred = predictor.run_prediction(res)
                        score = round(score[0], 2)
                        pred = "✅ Phê duyệt" if pred[0] == 0 else "🛑 Từ chối"
                        info = pd.DataFrame(
                            list(res.drop(columns=['Default'], errors='ignore').iloc[0].to_dict().items()), 
                            columns=["Column", "Value"]
                        )
                        shap_plot = predictor.plot_shap_waterfall(res)
                        return info, score, pred, shap_plot
                    
                    profile.change(
                        fn=lookup_fn,
                        inputs=[profile, df_state],
                        outputs=[profile_info, score_box, decision_box, expl_plot]
                    )

                    gr.Markdown("**Ghi chú của nhân viên**")
                    officer_note = gr.Textbox(label="Ghi chú (tùy chọn)", placeholder="Ghi chú cho hồ sơ")
                    save_note_btn = gr.Button("Lưu ghi chú", variant="secondary")
                    # status_note = gr.Textbox(label="Trạng thái ghi chú", interactive=False)
                    status_note = gr.Markdown("")

                    def _save_note(note):
                        return "Đã lưu." if note else "Ghi chú rỗng."
                    # save_note_btn.click(fn=_save_note, inputs=officer_note, outputs=status_note)
                    save_note_btn.click(fn=_save_note, inputs=officer_note, outputs=status_note)


        # -----------------------
        # Risk Manager Tab
        # -----------------------
        with gr.TabItem("C-level /  Risk Manager", visible=False) as tab_risk:
            gr.Markdown("#### Risk Overview • Thống kê & Cảnh báo")
            # Controls
            with gr.Row():
                with gr.Column():
                    upload_risk_btn = gr.File(label="Upload dataset", file_types=[".csv"])
                    df_state = gr.State()  # Save df into a Gradio state variable
                with gr.Column():    
                    start_date = gr.DateTime(label="Start Date", type="datetime")
                    end_date = gr.DateTime(label="End Date", type="datetime")
                    analyze_btn = gr.Button("Phân tích", variant="primary")
            
            # Charts
            with gr.Row():
                hist_plot = gr.Plot(label="Histogram Probability Score")
                pie_plot = gr.Plot(label="Pie Chart Predicted Label")
                weekly_plot = gr.Plot(label="Weekly Default Rate (%)")

            # Functions
            def _load_risk_dataset(file):
                if file is None:
                    return [None] * 3
                df = pd.read_csv(file.name)
                df["request_date"] = pd.to_datetime(df["request_date"])
                min_d = df["request_date"].min()
                max_d = df["request_date"].max()
                return df, min_d, max_d

            # Load date range into filters from data
            upload_risk_btn.upload(
                fn=_load_risk_dataset,
                inputs=upload_risk_btn,
                outputs=[df_state, start_date, end_date]
            )

            def analyze(df, start_d, end_d):
                if df is None:
                    return None, None, None
        
                df = df.copy()
                df = df[(df["request_date"] >= start_d) & (df["request_date"] <= end_d)]
                if len(df) == 0:
                    raise ValueError("Không có dữ liệu trong khoảng ngày đã chọn.")
        
                # Run model prediction
                pred_scores, pred_labels = predictor.run_prediction(df)
                df["pred_score"] = pred_scores
                df["pred_label"] = pred_labels
        
                return plot_eda_charts(df)

            analyze_btn.click(
                fn=analyze,
                inputs=[df_state, start_date, end_date],
                outputs=[hist_plot, pie_plot, weekly_plot]
            )
    

        # -----------------------
        # Business Analyst / Data Scientist Tab
        # -----------------------
        with gr.TabItem("BA / Data Analyst / Data Scientist", visible=False) as tab_ds:
            gr.Markdown("#### Giám sát mô hình & phân tích đặc trưng")
            with gr.Row():
                with gr.Column():
                    upload_eval_btn = gr.File(label="Upload dataset", file_types=[".csv"])
                    roc_plot = gr.Plot(label="ROC Curve")
                    cm_plot = gr.Plot(label="Confusion Matrix")
                with gr.Column():
                    metric_table = gr.Dataframe(label="Metrics")
                    fi_plot = gr.Plot(label="Feature Importance")
            
            def _run_model_eval(file):
                if file is None:
                    return None, None, None, None
                df = pd.read_csv(file.name)
                return predictor.run_evaluation(df)
            
            upload_eval_btn.upload(
                fn=_run_model_eval,
                inputs=upload_eval_btn,
                outputs=[roc_plot, cm_plot, metric_table, fi_plot]
            )

            gr.Markdown("---")
            gr.Markdown("**Notes / Observability**")
            drift_note = gr.Textbox(label="Drift warning", interactive=False, value="No significant feature drift detected in the last 30 days.")
            explain_note = gr.Textbox(label="Explainability tip", interactive=False, value="Use SHAP summary for cohort-level insights; check model behavior on low-income segment.")

        # -----------------------
        # End Users
        # -----------------------
        with gr.Tab("Người dùng cuối — Tra cứu kết quả", visible=False) as tab_user:
            gr.Markdown("### 🔍 Tra cứu kết quả vay vốn")
    
            with gr.Group():
                phone = gr.Textbox(label="Số điện thoại", placeholder="Nhập số điện thoại đã đăng ký", max_lines=1)
                send_otp_btn = gr.Button("Gửi mã OTP", variant="secondary")
                otp = gr.Textbox(label="Nhập OTP", placeholder="Nhập mã xác nhận gồm 6 chữ số")
                verify_btn = gr.Button("Xác thực & Tra cứu", variant="primary")
    
            result_status = gr.Textbox(label="Trạng thái tra cứu", interactive=False)
            user_info = gr.Dataframe(
                label="Kết quả tra cứu", interactive=False, wrap=True, 
                column_widths=["200px", "200px", "200px", "auto"],   # 3 cột đầu 200px, cột 4 auto
            )
    
            # --- Mock OTP backend ---
            import random
            otp_state = gr.State(value="")
    
            def _send_otp(phone):
                if not phone:
                    return "⚠️ Vui lòng nhập số điện thoại.", ""
                otp_code = f"{random.randint(100000, 999999)}"
                # (ở môi trường thật: gửi OTP qua SMS)
                return f"✅ OTP đã gửi đến {phone} (OTP: {otp_code})", otp_code
    
            send_otp_btn.click(fn=_send_otp, inputs=[phone], outputs=[result_status, otp_state])
    
            df_processed_user = pd.read_csv("../data/test_processed_user_sample.csv")
            df_processed_user["phone"] = df_processed_user["phone"].astype(str).str.zfill(10)
            def _verify_otp(otp_input, otp_expected, phone):
                if otp_input.strip() != otp_expected.strip():
                    return "❌ Mã OTP không đúng hoặc đã hết hạn.", pd.DataFrame()
                df_res = df_processed_user.query(f"phone == '{phone}'")
                df_res = (
                    df_res
                    .assign(process_result=lambda dfx: dfx.Default.apply(lambda x: "🛑 Từ chối" if x == 1 else "✅ Được phê duyệt"))
                    .loc[:, ["name", "process_date", "process_result", "explains"]]
                    .rename(columns={
                        "name": "Họ tên",
                        "process_date": "Duyệt lúc",
                        "process_result": "Kết quả",
                        "explains": "Lý do"
                    })
                )
                return "✅ Xác thực thành công!", df_res
    
            verify_btn.click(fn=_verify_otp, inputs=[otp, otp_state, phone], outputs=[result_status, user_info])

        
        # -----------------------
        # Login
        # -----------------------
        login_btn.click(login, [username, password], [login_status, role_state, login_box])
        role_state.change(show_tabs, [role_state], [tab_officer, tab_risk, tab_ds, tab_user])


    # Footer
    gr.Markdown("---")
    gr.Markdown("© Demo UI — Loan Default Prediction • Designed for coursework / prototype")

In [ ]:
app.launch()